In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, Sequential, clone_model, load_model
from tensorflow.keras.layers import Input, Dense, add, concatenate, Conv2D,Dropout,\
BatchNormalization, Flatten, MaxPooling2D, AveragePooling2D, Activation

In [ ]:
def load_CIFAR10_data(standarized = False, verbose = False):
    
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    
    y_train = np.squeeze(y_train)
    y_test = np.squeeze(y_test)
    # substract mean and normalized to [-1/2,1/2]
    if standarized: 
        X_train = X_train/255
        X_test = X_test/255
        mean_image = np.mean(X_train, axis=0)
        X_train -= mean_image
        X_test -= mean_image
        
    
    
    if verbose == True: 
        print("X_train shape :", X_train.shape)
        print("X_test shape :", X_test.shape)
        print("y_train shape :", y_train.shape)
        print("y_test shape :", y_test.shape)
    
    return X_train, y_train, X_test, y_test

def generate_imcomplete_data(X, y, missing_digits = None, verbose = False):
    if missing_digits is None:
        idx = np.ones_like(y, dtype = bool)
    else:
        idx = [y != i for i in missing_digits]
        idx = np.all(idx, axis = 0)
    X_incomplete, y_incomplete = X[idx], y[idx]
    if verbose == True:
        print("X shape :", X_incomplete.shape)
        print("y shape :", y_incomplete.shape)
    return X_incomplete, y_incomplete

In [ ]:
X_train, y_train, X_test, y_test = load_CIFAR10_data(standarized = False, verbose = True)

In [ ]:
model_D, x = None, None
x = Input(shape = (32,32,3))
y = Conv2D(filters = 128, kernel_size = (2,2), strides = 2, padding = "same", 
          activation = None)(x)
y = BatchNormalization()(y)
y = Activation("relu")(y)
y = MaxPooling2D(pool_size = (2,2), strides = 2, padding = "same")(y)

y = Flatten()(y)
y = Dense(units = 10, activation = "softmax")(y)

model_D = Model(inputs = x, outputs = y)

model_D.compile(optimizer=tf.keras.optimizers.Adam(lr = 1e-3), 
                      loss = "sparse_categorical_crossentropy",
                      metrics = ["accuracy"])

In [ ]:
model_D.summary()

In [ ]:
model_D.fit(X_train, y_train, batch_size=128, epochs = 10, shuffle=False, verbose = 1, 
               validation_data=(X_test, y_test))